In [1]:
! pip install simpy

In [2]:
import random
import simpy

In [3]:
RANDOM_SEED = 0         
NUM_COUNTERS = 3        
TIME_CONSUMING = 5      
TIME_INTERVAL = 5       
SIM_TIME = 100        
CLIENT_NUMBER = 2 

In [12]:
class Hotel(object):

    def __init__(self, env, num_counters, checkintime):
        self.env = env
        self.counters = simpy.Resource(env, num_counters)
        self.checkintime = checkintime
        self.allClient = 0
        self.accomplishClient = 0
 
    def checkin(self, people):
             
        yield self.env.timeout(random.randint(3, 10))  
        self.allClient += 1
        per = random.randint(50, 99)
        print("%s's The completion of task：%d%%." % (people, per))
        
        if per > 80:
            self.accomplishClient += 1
 
        print("Number of clients served by counters：%d, "
              "Hotel service compliance rate：%.2f。" % (self.allClient, float(self.accomplishClient) / float(self.allClient)))
    
        print(f'-----------------------------------------------------------------------------')
    

In [13]:
def Client(env, name, cw):
 
    print('%s Arriving at the hotel at %.2f.' % (name, env.now))
    with cw.counters.request() as request:
        yield request
        print('%s Check in at %.2f.' % (name, env.now))
        yield env.process(cw.checkin(name))
        print('%s Leaving the hotel at %.2f.' % (name, env.now))
        

In [14]:
def setup(env, num_counters, checkintime, t_inter, clientNumber):
   
    hotel = Hotel(env, num_counters, checkintime)
  
    for i in range(clientNumber):
        env.process(Client(env, 'Client_%d' % i, hotel))
 
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 3))
        i += 1
        env.process(Client(env, 'Client_%d' % i, hotel))
 

In [15]:
print('Start simulation!')
 
random.seed()



env = simpy.Environment()
env.process(setup(env, NUM_COUNTERS, TIME_CONSUMING, TIME_INTERVAL, CLIENT_NUMBER))

env.run(until=SIM_TIME)

Start simulation!
Client_0 Arriving at the hotel at 0.00.
Client_1 Arriving at the hotel at 0.00.
Client_0 Check in at 0.00.
Client_1 Check in at 0.00.
Client_2 Arriving at the hotel at 3.00.
Client_2 Check in at 3.00.
Client_1's The completion of task：75%.
Number of clients served by counters：1, Hotel service compliance rate：0.00。
-----------------------------------------------------------------------------
Client_1 Leaving the hotel at 6.00.
Client_0's The completion of task：56%.
Number of clients served by counters：2, Hotel service compliance rate：0.00。
-----------------------------------------------------------------------------
Client_0 Leaving the hotel at 7.00.
Client_2's The completion of task：87%.
Number of clients served by counters：3, Hotel service compliance rate：0.33。
-----------------------------------------------------------------------------
Client_2 Leaving the hotel at 10.00.
Client_3 Arriving at the hotel at 11.00.
Client_3 Check in at 11.00.
Client_4 Arriving at the

Within the specified time, this simulates the process of how many clients can check in according to the number of staff provided by the hotel. In addition, each client will give a score to the staff. Therefore, this simulation requires real-time statistics to calculate the task completion and service compliance rate.


1.The number of counter staff is limited, but the check-in time for each client is different.\
2.When the client arrives at the hotel, if there is a free counter staff, it will immediately accept the service; if not, it will wait until the other counter staff is free. 
3.The interval time for each guest to the hotel is random. 